In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import pandas as pd
import requests
from io import StringIO
import re
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
import umap.umap_ as umap
import matplotlib.pyplot as plt
import hdbscan
pd.set_option('display.max_colwidth', None)

In [2]:
sheet_name_lsit = ['Sci/2', 'Sci_Extra/1', 'Sci/1', 'Sci_Extra/2', 'Engineering/1', 'Engineering_Extra/1', 'Engineering/2', 'Engineering_Extra/2', 'SIIT/2', 'SIIT/1', 'Innovation_Extra/1', 'Innovation_Extra/2', 'CIS_Extra/1', 'CIS/1', 'CIS_Extra/2', 'CIS/2']

In [3]:
sheet_id = "1ExtEUwVLi5Y_mmqxNd0oX7WrWzj8n6TeBLcQg7Xu0GU"
dataframe = pd.DataFrame(columns=["code", "desc", "add1", "add2", "add3","ma"])

for sheet_name in sheet_name_lsit:
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    response = requests.get(url)
    decoded_content = response.content.decode('utf-8')
    data_tem =  pd.read_csv(StringIO(decoded_content))
    dataframe = pd.concat([dataframe, data_tem], ignore_index=True)

print(dataframe.shape)
for i in dataframe.columns:
    dataframe[f"{i}"] = dataframe[f"{i}"].str.strip().str.lower()


print(dataframe.shape)

(4272, 6)
(4272, 6)


In [4]:
dataframe["desc"] = dataframe["desc"].str.replace(r"[.,]", "", regex=True).str.strip()

dataframe.drop_duplicates(subset=["code"],inplace=True)
dataframe.dropna(subset=["desc","code"],inplace=True)

# df = dataframe[dataframe.duplicated(subset=["desc"], keep=False)]
# print(df.desc.head(3))
# print(df.shape)
# dataframe.drop_duplicates(subset=["desc"], keep=False,inplace=True)

# df["code"] = df.groupby("desc")["code"].transform(lambda x: ", ".join(x))
# df = df.drop_duplicates(subset="desc")
# print(df.shape)
# dataframe = pd.concat([df,dataframe])

In [5]:
dataframe["add1"] = dataframe["add1"].str.replace("คพ","cs")
dataframe["add1"] = dataframe["add1"].str.replace("ทก","at")
dataframe["add1"] = dataframe["add1"].str.replace("คป","am")
dataframe["add1"] = dataframe["add1"].str.replace("ทช","bt")
dataframe["add1"] = dataframe["add1"].str.replace("วท","sc")
dataframe["add1"] = dataframe["add1"].str.replace("คม","cm")
dataframe["add1"] = dataframe["add1"].str.replace("ค","ma")
dataframe["add1"] = dataframe["add1"].str.replace("ส","st")

dataframe["add2"] = dataframe["add2"].str.replace("คพ","cs")
dataframe["add2"] = dataframe["add2"].str.replace("ทก","at")
dataframe["add2"] = dataframe["add2"].str.replace("คป","am")
dataframe["add2"] = dataframe["add2"].str.replace("ทช","bt")
dataframe["add2"] = dataframe["add2"].str.replace("วท","sc")
dataframe["add2"] = dataframe["add2"].str.replace("คม","cm")
dataframe["add2"] = dataframe["add2"].str.replace("ค","ma")
dataframe["add2"] = dataframe["add2"].str.replace("ส","st")


In [6]:
def extract_courses(row):
    pattern = r"[A-Za-zก-ฮ]{1,2}\.? ?\d{3}"
    listt = []
    if isinstance(row, str):
        matches = re.findall(pattern, row)  
        listt.extend(matches)
        return listt
    return None 

dataframe["cancel_course"] = dataframe["add1"].apply(extract_courses) + dataframe["add2"].apply(extract_courses)

In [7]:
dataframe.drop("add3",inplace=True,axis=1)

In [10]:
dataframe.to_csv("data1.csv")

In [9]:
dataframe.rename(columns={
    "desc":"description",
    "add1":"disciplinary course(s)",
    "add2":"consecutive course(s)",
    "ma":"faculty"
    
},inplace=True)